In [31]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle, Ellipse, FancyBboxPatch
import numpy as np
%matplotlib inline

import pcbnew

fname_buck = "buck-module.kicad_pcb"
fname_digi = "../digitizer2.kicad_pcb"
pcb = pcbnew.LoadBoard(fname_digi)

In [52]:
def create_board_figure(pcb, highlight_refs=[]):
    plt.figure(figsize=(5.8, 8.2))
    ax = plt.subplot("111", aspect="equal")
    
    color_pad1 = "lightgray"
    color_pad2 = "#AA0000"
    color_bbox1 = "None"
    color_bbox2 = "#E9AFAF"

    # get board edges (assuming rectangular, axis aligned pcb)
    edge_coords = []
    for d in pcb.GetDrawings():
        if (d.GetLayer() == pcbnew.Edge_Cuts):
            edge_coords.append(d.GetStart())
            edge_coords.append(d.GetEnd())
    edge_coords = np.asarray(edge_coords) * 1e-6
    board_xmin, board_ymin = edge_coords.min(axis=0)
    board_xmax, board_ymax = edge_coords.max(axis=0)

    # draw board edges
    rct = Rectangle((board_xmin, board_ymin), board_xmax-board_xmin, board_ymax-board_ymin, angle=0)
    rct.set_color("None")
    rct.set_edgecolor(None)
    rct.set_linewidth(3)
    ax.add_patch(rct)

    # draw parts
    centers = []
    for m in pcb.GetModules():
        if m.GetLayer() != pcbnew.F_Cu:
            continue
        ref, center = m.GetReference(), np.asarray(m.GetCenter()) * 1e-6
        highlight = ref in highlight_refs
        centers.append(center)

        # bounding box
        mrect = m.GetFootprintRect()
        mrect_pos = np.asarray(mrect.GetPosition()) * 1e-6
        mrect_size = np.asarray(mrect.GetSize()) * 1e-6
        rct = Rectangle(mrect_pos, mrect_size[0], mrect_size[1])
        rct.set_color(color_bbox2 if highlight else color_bbox1)
        rct.set_zorder(-1)
        if highlight:
            rct.set_linewidth(.1)
            rct.set_edgecolor(color_pad2)
        ax.add_patch(rct)

        # plot pads
        for p in m.Pads():
            pos = np.asarray(p.GetPosition()) * 1e-6
            size = np.asarray(p.GetSize()) * 1e-6 *.9

            # TODO: check shape and offset
            shape = p.GetShape()
            offset = p.GetOffset()

            # pad rect
            angle = p.GetOrientation() * 0.1
            cos, sin = np.cos(np.pi/180.*angle), np.sin(np.pi/180.*angle)
            dpos = np.dot([[cos, -sin], [sin, cos]], -.5*size)

            if shape == 1:
                rct = Rectangle(pos + dpos, size[0], size[1], angle=angle)
            elif shape == 2:
                rct = Rectangle(pos + dpos, size[0], size[1], angle=angle)
            elif shape == 0:
                rct = Ellipse(pos, size[0], size[1], angle=angle)
            else:
                print("Unsupported pad shape")
                continue
            rct.set_color(color_pad2 if highlight else color_pad1)
            rct.set_zorder(1)
            ax.add_patch(rct)

        # draw text
        #plt.text(center[0]+0.2, center[1]-.2, ref)

    centers = np.asfarray(centers)
    #plt.plot(centers[:, 0], centers[:, 1], ",")

    plt.xlim(board_xmin, board_xmax)
    plt.ylim(board_ymax, board_ymin)

    plt.axis('off')
    #plt.tight_layout(0)

In [83]:
bom_table = """
1	XC7A35T	BGA-256_pitch1mm_dia0.4mm	U1
1	FT2232H-56Q	VQFN-FT2232H-56Q_pitch0.5	U2
1	ADS5403	BGA-196_pitch0.8mm_dia0.4mm	U4
1	MT41J128M16	BGA-96_pitch0.8mm_dia0.35mm	U3
1	TPS79618DCQ	SOT-223-6	U6
1	TPS79633DCQ	SOT-223-6	U7
8	4.7k	R_0402	R2, R1, R21, R23, R26, R24, R6, R10
1	93LC46BT-I/OT	SOT-23-6	U8
4	10k	R_0603	R4, R7, R8, R3
1	12k	R_0603	R9
23	4.7uF	C_0805	C9, C5, C3, C41, C42, C47, C48, C57, C58, C64, C65, C71, C72, C123, C126, C125, C122, C128, C130, C140, C141, C10, C7
1	2.2k	R_0603	R5
42	0.1uF	C_0402	C1, C11, C13, C14, C15, C16, C17, C22, C12, C8, C33, C34, C80, C81, C82, C83, C84, C85, C92, C93, C94, C95, C96, C97, C104, C105, C106, C107, C108, C109, C116, C117, C118, C119, C120, C121, C132, C134, C146, C148, C2, C18
2	18pF	C_0603	C4, C6
8	ferrite	R_0805	L1, L2, L9, L10, L11, L12, L13, L14
1	USB_OTG	USB_Micro-B	P1
2	100	R_0402	R16, R19
2	1k	R_0402	R17, R18
3	1uF	C_0402	C35, C144, C147
33	10nF	C_0402	C37, C36, C74, C75, C76, C77, C78, C79, C86, C87, C88, C89, C90, C91, C98, C99, C100, C101, C102, C103, C110, C111, C112, C113, C114, C115, C124, C127, C133, C135, C143, C159, C19
1	240	R_0402	R20
23	0.47uF	C_0402	C38, C39, C40, C44, C45, C46, C51, C53, C54, C55, C56, C60, C61, C62, C63, C67, C68, C69, C70, C136, C137, C138, C139
11	47uF	C_1210	C43, C49, C50, C59, C66, C73, C129, C131, C142, C155, C156
1	JUMPER3	Pin_Header_Straight_1x03	JP2
2	SN65LVDS100DGK	TSSOP-8_3x3mm_Pitch0.65mm	U10, U11
2	50	R_0603	R25, R22
2	1.8V	LED_0603	D1, D2
2	750	R_0603	R27, R28
2	ETC1-1-13	ETC1-1-13	T1, T2
3	SMA	SMA_ANGLE_THT	P4, P5, P6
8	0	R_0603	R32, R30, R29, R37, R38, R11, R12, R13
1	6.8pF NC	C_0603	C145
2	25	R_0603	R31, R33
1	100 NC	R_0402	R34
2	NC	R_0603	R35, R36
1	CONN_02X03	Pin_Header_Angled_2x03	P7
1	ASG2-D-X-A-500.000MHZ	ASG2	U12
1	CONN_02X03	Pin_Header_Straight_2x03	P8
1	FXO-LC526R-100	FXO-LC52	U16
1	ABM8G-12.000MHZ-18-D2Y-T 	ABM8G	Y1
2	220uF	c_elec_5x5.7	C154, C20
1	TPS79625DCQ	SOT-223-6	U5
"""
lines = (line for line in bom_table.splitlines() if line)
bom_table = (line.split("\t") for line in lines)
def sort_func(row):
    qty, value, footpr, refs = row
    conn = "PJ".find(refs[0]) != -1
    return (conn, -int(qty))
bom_table = sorted(bom_table, key=sort_func)

In [87]:
from matplotlib.backends.backend_pdf import PdfPages
with PdfPages('test.pdf') as pdf:
    for row in bom_table:
        qty, value, footpr, refs = row
        highlight_refs = set((r.strip() for r in refs.split(", ")))
        
        create_board_figure(pcb, highlight_refs)
        plt.title("%s, %s" % (value, footpr))
        pdf.savefig()
        plt.close()